In [2]:
'''
    AlexNet with weights from Caffe using https://github.com/ethereon/caffe-tensorflow
    The weights are loaded from "models/alexNet.npy" and a provided image is tested using the loaded weights. 
    Code to save the model and meta-graph into TensorFlow format is also available. Just call saveAsCkptFile()
    
    AlexNet composition:
        conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
        lrn(2, 2e-05, 0.75, name='norm1')
        max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
        conv(5, 5, 256, 1, 1, group=2, name='conv2')
        lrn(2, 2e-05, 0.75, name='norm2')
        max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
        conv(3, 3, 384, 1, 1, name='conv3')
        conv(3, 3, 384, 1, 1, group=2, name='conv4')
        conv(3, 3, 256, 1, 1, group=2, name='conv5')
        fc(4096, name='fc6')
        fc(4096, name='fc7')
        fc(1000, relu=False, name='fc8')
        softmax(name='prob'))     
        
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

import os
import numpy as np
from scipy.misc import imread

from classes import labels

IMAGE = "data/image.jpg"
MODEL = "models/alexNet.npy"
MODEL_CKPT = "models/alexNet.ckpt"

WIDTH = 227
HEIGHT = 227
COLOR_CHANNELS = 3
NUMBER_OF_CLASSES = 1000

trainingX = np.zeros((1, WIDTH, HEIGHT, COLOR_CHANNELS)).astype(np.float32).shape[1:]
trainingY = np.zeros((1, NUMBER_OF_CLASSES)).shape[1]

def readImage(filePath):
    i = (np.random.random((1,)+ trainingX)/255.).astype(np.float32)
    image = i.copy()
    image[0,:,:,:] = (imread(filePath)[:,:,:3]).astype(np.float32)
    image = image-np.mean(image)
    
    x = tf.Variable(image)
    return x

def saveAsCkptFile():
    '''
        Save the model in TensorFlow format
        If you want to work with this model format, in alexNetCkpt.ipynb is shown how to load the ckpt file
    ''' 
    saver = tf.train.Saver()
    savePath = saver.save(sess, MODEL_CKPT)
    print("Model saved in file: %s" % savePath)
    

x = readImage(IMAGE)
net = np.load(MODEL).item()

def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w, name, padding="VALID", group=1):
    '''
        Partly taken from kaffe/tensorflow/network.py (https://github.com/ethereon/caffe-tensorflow)
        Copyright (c) 2016 Saumitro Dasgupta
    '''
    c_i = input.get_shape()[-1]
    assert c_i%group==0
    assert c_o%group==0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding, name=name)
    
    if group==1:
        conv = convolve(input, kernel)
    else:
        input_groups = tf.split(3, group, input)
        kernel_groups = tf.split(3, group, kernel)
        output_groups = [convolve(i, k) for i,k in zip(input_groups, kernel_groups)]
        conv = tf.concat(3, output_groups)
    return  tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape().as_list())

#conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
conv1 = conv(x, tf.Variable(net["conv1"][0]), tf.Variable(net["conv1"][1]), 11, 11, 96, 4, 4, name='conv1', padding="SAME", group=1)
conv1 = tf.nn.relu(conv1)

#lrn(2, 2e-05, 0.75, name='norm1')
lrn1 = tf.nn.local_response_normalization(conv1, depth_radius=2, alpha=2e-05, beta=0.75, bias=1.0, name='norm1')

#max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool1')

#conv(5, 5, 256, 1, 1, group=2, name='conv2')
conv2 = conv(maxpool1, tf.Variable(net["conv2"][0]), tf.Variable(net["conv2"][1]), 5, 5, 256, 1, 1, name='conv2', padding="SAME", group=2)
conv2 = tf.nn.relu(conv2)

#lrn(2, 2e-05, 0.75, name='norm2')
lrn2 = tf.nn.local_response_normalization(conv2, depth_radius=2, alpha=2e-05, beta=0.75, bias=1.0, name='norm2')

#max_pool(3, 3, 2, 2, padding='VALID', name='pool2')                                                  
maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool2')

#conv(3, 3, 384, 1, 1, name='conv3')
conv3 = conv(maxpool2, tf.Variable(net["conv3"][0]), tf.Variable(net["conv3"][1]), 3, 3, 384, 1, 1, name='conv3', padding="SAME", group=1)
conv3 = tf.nn.relu(conv3)

#conv(3, 3, 384, 1, 1, group=2, name='conv4')
conv4 = conv(conv3, tf.Variable(net["conv4"][0]), tf.Variable(net["conv4"][1]), 3, 3, 384, 1, 1, name='conv4', padding="SAME", group=2)
conv4 = tf.nn.relu(conv4)

#conv(3, 3, 256, 1, 1, group=2, name='conv5')
conv5 = conv(conv4, tf.Variable(net["conv5"][0]), tf.Variable(net["conv5"][1]), 3, 3, 256, 1, 1, name='conv5', padding="SAME", group=2)
conv5 = tf.nn.relu(conv5)

#max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
maxpool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool5')

#fc(4096, name='fc6')
fc6 = tf.nn.relu_layer(tf.reshape(maxpool5, [1, int(np.prod(maxpool5.get_shape()[1:]))]), tf.Variable(net["fc6"][0]), tf.Variable(net["fc6"][1]), name='fc6')

#fc(4096, name='fc7')
fc7 = tf.nn.relu_layer(fc6, tf.Variable(net["fc7"][0]), tf.Variable(net["fc7"][1]), name='fc7')

#fc(1000, relu=False, name='fc8')
fc8 = tf.nn.xw_plus_b(fc7, tf.Variable(net["fc8"][0]), tf.Variable(net["fc8"][1]), name='fc8')

#softmax(name='prob'))
prob = tf.nn.softmax(fc8, name='prob')

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

prediction = sess.run(prob)

# saveAsCkptFile()

indices = np.argsort(prediction)[0,:][::-1] # the entry with the highest probability first
for i in range(5):
    print(labels[indices[i]], prediction[0, indices[i]]) # print out the five entries with the highest probabilities

Model saved in file: alexNet.ckpt
kelpie 0.486945
Cardigan, Cardigan Welsh corgi 0.191103
Border collie 0.171735
collie 0.0684693
German shepherd, German shepherd dog, German police dog, alsatian 0.0169954
